<h1>Get the player injury logs</h1>
<p>The following code looks at the injury lists and scrapes them off <a href="http://www.prosportstransactions.com">prosportstransactions.com</a>.  It will take further significant work to turn them into useful datapoints.</p>

In [ ]:
import pandas as pd
import numpy as np
import requests as r
import re
from bs4 import BeautifulSoup as bs

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
#import player list from csv and tidy 
players = pd.read_csv('nhlplayerlist_goalieonly.txt','\t')
players.drop(columns=['Unnamed: 0','index'],axis=1,inplace=True)
players.head(5)
#add a new injuries_found column to the list
players['injuries_found'] = np.nan

<h2>Grab the injury list</h2>
<p>For each player, grab the table of injuries off of <a href="http://www.prosportstransactions.com">prosportstransactions.com</a>.  This dataframe is then saved to a new folder.  Additionally, the player list is updated denoting players without injury reports or if they have them,the length of the table.  Later on I will need to cycle back and reformat the injury report.</p>

In [ ]:
#for each player in the list
for i,player in players.iterrows():

    pageno = 0 #current page no
    page_max = 99 #total number of pages
    injurytable = pd.DataFrame()

    while pageno <= page_max:
        #website - check for Craig Anderson
        url= 'http://www.prosportstransactions.com/hockey/Search/SearchResults.php?Player=%s&Team=&BeginDate=1990-09-01&EndDate=&ILChkBx=yes&InjuriesChkBx=yes&submit=Search&start=%d' %(player['player'],pageno*25)
        page = r.get(url)
        soup = bs(page.text,'html.parser')
        table = soup.find_all('tr')
        #check to see if there's any data?
        if len(table)==0:
            players.loc[i,'injuries_found'] = 0
            break
        else:
            for ri,row in enumerate(table):
                rowe = []
                #get data
                for c in row.find_all('td'):
                    rowe.append(c.text)

                #check if we're on the final row of the page
                if (rowe[0]=='\xa0'):
                    if pageno==0:
                        page_max=int(rowe[2].split()[-1])-1 #set the maximum number of pages (only the first time)
                else:
                    if (ri==0): #its the header of the table
                        if pageno==0: #import the first time
                            injurytable = pd.DataFrame(columns=rowe)
                        else:
                            pass #burn the repeated table headers
                    else:
                        injurytable = injurytable.append(dict(zip(injurytable.columns,rowe)),ignore_index=True)
            pageno += 1
    players.loc[i,'injuries_found'] = injurytable.shape[0]
    if injurytable.shape[0]>0:
        injurytable.to_csv('player_injurylist\\'+player['unique_id']+'.txt','\t')
    
players.to_csv('nhlplayerlist_goalies.txt','\t')

In [ ]:
print('There are injuries reported for a total of %s of %s players' % (players[players['injuries_found']>0].count()[0],players.shape[0]))